# 2026 World Cup Draw Simulator

The purpose of this notebook is to write out code that can be used to simulate the 2026 FIFA World Cup draw. 

In [1]:
import random
from collections import defaultdict

GROUPS = [chr(c) for c in range(ord('A'), ord('L') + 1)]  # A-L

In [ ]:
# ---------------------------
# Team data (as given by you)
# ---------------------------
POT1 = ["USA", "Mexico", "Canada", "Spain", "Argentina", "France", "England",
        "Portugal", "Brazil", "Netherlands", "Belgium", "Germany"]

POT2 = ["Croatia", "Morocco", "Colombia", "Uruguay", "Switzerland", "Senegal",
        "Austria", "Japan", "Iran", "South Korea", "Ecuador", "Australia"]

POT3 = ["Scotland", "Panama", "Norway", "Egypt", "Algeria", "Paraguay",
        "Tunisia", "Ivory Coast", "Costa Rica", "Uzbekistan", "Qatar", "South Africa"]

POT4 = ["Saudi Arabia", "Jordan", "Cape Verde", "Jamaica", "Ghana", "New Zealand",
        "Italy", "Turkiye", "Ukraine", "Wales", "Nigeria", "Bolivia"]

In [3]:
# ---------------------------
# Confederations
# ---------------------------
UEFA = {
    "Spain", "France", "England", "Portugal", "Netherlands", "Belgium", "Germany",
    "Croatia", "Denmark", "Switzerland",
    "Norway", "Austria", "Turkiye", "Ukraine", "Wales", "Italy"
}

CONMEBOL = {"Argentina", "Brazil", "Colombia", "Uruguay", "Ecuador", "Paraguay", "Bolivia"}
CONCACAF = {"USA", "Mexico", "Canada", "Panama", "Costa Rica", "Jamaica"}
CAF = {"Morocco", "Senegal", "Tunisia", "Ivory Coast", "Algeria", "Egypt",
       "Ghana", "Nigeria", "Cape Verde", "South Africa"}
AFC = {"Japan", "Iran", "South Korea", "Australia", "Uzbekistan", "Qatar",
       "Saudi Arabia", "Jordan"}
OFC = {"New Zealand"}

CONFEDS = {}
for t in UEFA: CONFEDS[t] = "UEFA"
for t in CONMEBOL: CONFEDS[t] = "CONMEBOL"
for t in CONCACAF: CONFEDS[t] = "CONCACAF"
for t in CAF: CONFEDS[t] = "CAF"
for t in AFC: CONFEDS[t] = "AFC"
for t in OFC: CONFEDS[t] = "OFC"

# sanity check: all teams covered
ALL_TEAMS = set(POT1 + POT2 + POT3 + POT4)
missing_confed = [t for t in ALL_TEAMS if t not in CONFEDS]
if missing_confed:
    raise RuntimeError(f"Missing confed mapping for: {missing_confed}")

In [4]:
# ---------------------------
# Helpers
# ---------------------------

def allowed_in_group(group_teams, team):
    """
    Confederation constraint:
      - No two from same confed in a group EXCEPT UEFA (UEFA can repeat).
    """
    team_conf = CONFEDS[team]
    if team_conf == "UEFA":
        return True  # UEFA exception per prompt
    # Non-UEFA confederations may not repeat in a group
    return all(CONFEDS[t] != team_conf for t in group_teams)

def groups_state_ok(groups):
    """Check each group has at most 4 members and constraints respected."""
    for g in GROUPS:
        if len(groups[g]) > 4:
            return False
        # Verify constraints
        confeds_present = defaultdict(int)
        for t in groups[g]:
            c = CONFEDS[t]
            if c != "UEFA":
                confeds_present[c] += 1
                if confeds_present[c] > 1:
                    return False
    return True

def pretty_print_groups(groups):
    for g in GROUPS:
        print(f"Group {g}: " + ", ".join(groups[g]))

In [5]:
# ---------------------------
# Drawing mechanics
# ---------------------------

def assign_pot1_with_hosts(seed=None):
    """
    Pre-draw hosts into A, B, D as specified.
    Randomly place remaining Pot 1 teams into remaining groups.
    """
    rng = random.Random(seed)

    groups = {g: [] for g in GROUPS}

    # Fixed hosts:
    # Mexico -> A, Canada -> B, USA -> D
    groups["A"].append("Mexico")
    groups["B"].append("Canada")
    groups["D"].append("USA")

    remaining_pot1 = [t for t in POT1 if t not in {"Mexico", "Canada", "USA"}]
    rng.shuffle(remaining_pot1)

    # Fill remaining groups (C, E-L) then whichever of A,B,D that aren't already full
    target_order = [g for g in GROUPS if len(groups[g]) == 0] + \
                   [g for g in GROUPS if len(groups[g]) == 1]  # A,B,D show here with one team

    idx = 0
    for g in target_order:
        while len(groups[g]) < 1 and idx < len(remaining_pot1):
            # Pot 1: exactly one team (or host already placed counts)
            # For groups with no host, add one Pot 1 team
            groups[g].append(remaining_pot1[idx])
            idx += 1

    # In 12 groups, each must have exactly one Pot 1 team
    return groups

def candidate_groups_for_team(groups, team):
    """Return list of groups (A-L) that can accept 'team' right now."""
    cands = []
    for g in GROUPS:
        if len(groups[g]) < 4 and allowed_in_group(groups[g], team):
            cands.append(g)
    return cands

def backtrack_assign_pot(groups, teams, rng):
    """
    Backtracking to place all 'teams' into 'groups' given current state.
    Uses MRV: pick the team with fewest candidate groups first.
    """
    if not teams:
        return True  # success

    # Pick team with minimum remaining valid groups
    teams_sorted = sorted(teams, key=lambda t: len(candidate_groups_for_team(groups, t)))
    team = teams_sorted[0]
    remaining = [t for t in teams if t != team]

    candidates = candidate_groups_for_team(groups, team)
    rng.shuffle(candidates)  # randomize among equals

    for g in candidates:
        groups[g].append(team)
        if groups_state_ok(groups) and backtrack_assign_pot(groups, remaining, rng):
            return True
        groups[g].pop()  # undo

    return False  # no placement worked

def simulate_draw(seed=None, verbose=False):
    """
    Perform one full simulation of the draw given the rules.
    Returns a dict: { 'A': [..4 teams..], ..., 'L': [..4 teams..] }
    """
    rng = random.Random(seed)

    # 1) Pot 1 with hosts fixed
    groups = assign_pot1_with_hosts(seed=rng.randint(0, 10**9))

    # 2) Pots 2 to 4 (full backtracking per pot)
    for pot in (POT2, POT3, POT4):
        teams = pot[:]  # copy
        rng.shuffle(teams)
        ok = backtrack_assign_pot(groups, teams, rng)
        if not ok:
            # if somehow fails (rare with these inputs), retry with a new shuffle
            # You can loop a few times; usually one pass works with MRV
            for _ in range(20):
                for g in groups:
                    # keep already assigned pots; undo only the current pot’s additions
                    # easiest approach: recompute groups from Pot1 + already placed pots
                    pass
            # Simpler: restart entire simulation with a new seed
            return simulate_draw(seed=rng.randint(0, 10**9), verbose=verbose)

    if verbose:
        pretty_print_groups(groups)
    return groups

In [6]:
# ---------------------------
# Run a sample simulation
# ---------------------------
if __name__ == "__main__":
    # Set a seed for reproducibility, or set to None for fresh random each run
    draw_seed = 42
    groups = simulate_draw(seed=draw_seed, verbose=True)

Group A: Mexico, South Africa, Norway, Turkiye
Group B: Canada, Senegal, Jordan, Italy
Group C: Belgium, Uzbekistan, Egypt, Bolivia
Group D: USA, Iran, Morocco, Ukraine
Group E: Argentina, Croatia, Qatar, Austria
Group F: Brazil, Australia, Costa Rica, Ghana
Group G: France, Denmark, Ivory Coast, Saudi Arabia
Group H: Germany, Colombia, Switzerland, New Zealand
Group I: England, Uruguay, Panama, Cape Verde
Group J: Portugal, Japan, Paraguay, Tunisia
Group K: Netherlands, Ecuador, South Korea, Algeria
Group L: Spain, Nigeria, Jamaica, Wales
